In [1]:
import pandas as pd

In [2]:
current_data = pd.read_csv("training_data/current_target.csv", index_col=0)

0.0           2.990610e+13
3e-08         1.509400e+14
5.8e-08       5.712310e+14
1.4e-07       1.109120e+15
2.8e-07       2.100110e+14
3.5e-07       1.802850e+14
6.25e-07      5.434490e+14
4e-06         7.206980e+14
4.8052e-05    1.171560e+15
0.00553       1.533240e+15
0.821         2.412640e+14
2.231         1.193780e+14
Name: 0, dtype: float64